In [2]:
import re
import datetime as dt
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
res = requests.get('https://www.moneytimes.com.br/tag/carteira-recomendada/')
soup = BeautifulSoup(res.content.decode())

#### Definição da heurística para pegar o mês atual

In [1]:
def has_current_month(s: str):
    map_months = {
        1: 'janeiro',
        2: 'fevereiro',
        3: 'março',
        4: 'abril',
        5: 'maio',
        6: 'junho',
        7: 'julho',
        8: 'agosto',
        9: 'setembro',
        10: 'outubro',
        11: 'novembro',
        12: 'dezembro'
    }
    
    current_month = map_months[dt.datetime.now().month]
    
    return current_month in s.lower()

In [6]:
links_carteiras = [s.find('a')['href'] for s in soup.find_all(class_='news-item__title')
                   if s.find('a').get_text().startswith('Dividendos:')
                   and has_current_month(s.find('a').get_text())]

In [8]:
links_carteiras

[]

In [15]:
for l in links_carteiras:
    print(l)

https://www.moneytimes.com.br/dividendos-as-melhores-acoes-de-marco-na-opiniao-de-20-analistas/


In [18]:
def get_dfs_from_link(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.content.decode())

    dfs = []
    for table in soup.find_all('table'):
        table_headers = [th.text for th in table.find_all('th')]
        table_data = [th.text for th in table.find_all('td')]
        table_rows = [table_data[i:i+len(table_headers)] for i in range(0, len(table_data), len(table_headers))]

        dfs.append(pd.DataFrame(table_rows, columns=table_headers))
    return dfs


In [24]:
for df in get_dfs_from_link(links_carteiras[0]):
    print(df)

           Empresas Indicações
0  1º          Vale         12
1  2º  Engie Brasil         11
2  2º         Taesa         11
3  3º          Vivo         10
                Empresas Indicações
0                   Vale         12
1           Engie Brasil         11
2                  Taesa         11
3                   Vivo         10
4          BB Seguridade          9
5     Energias do Brasil          6
6                 Itaúsa          6
7   Transmissão Paulista          6
8                     B3          4
9        Banco do Brasil          4
10              Bradesco          4
11                  Cesp          4
12                Copasa          4
13             AES Tietê          3
14                   CCR          3
15         Itaú Unibanco          3
16               Sanepar          3
17                Alupar          2
18              Banrisul          2
19             Bradespar          2
20                 Copel          2
21                  CPFL          2
22               

In [21]:
top5 = get_dfs_from_link(links_carteiras[0])[-1].loc[:4]

In [22]:
top5

,Empresas,Indicações
0,Vale,12
1,Engie Brasil,11
2,Taesa,11
3,Vivo,10
4,BB Seguridade,9
